# it is a common problem, notebook is for testing purposes so this code was generated with llm 

In [2]:
! pip install psycopg2-binary

In [ ]:
import psycopg2
import sys


def connect_to_db():
    """Establishes a connection to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(
            dbname="database-instance",
            user="testuje-sobie",
            password="",
            host="34.140.62.43",
            port="5432",  # Default PostgreSQL port
        )
        return conn
    except psycopg2.OperationalError as e:
        print(f"Unable to connect to the database: {e}")
        sys.exit(1)  # Exits the script if connection fails
    except Exception as e:
        print(f"An unexpected error occurred during connection: {e}")
        sys.exit(1)


def create_table_if_not_exists(conn):
    """Creates a sample table if it doesn't already exist."""
    create_table_query = """
    CREATE TABLE IF NOT EXISTS employees (
        id SERIAL PRIMARY KEY,
        first_name VARCHAR(50) NOT NULL,
        last_name VARCHAR(50) NOT NULL,
        email VARCHAR(100) UNIQUE,
        hire_date DATE DEFAULT CURRENT_DATE
    );
    """
    try:
        with conn.cursor() as cur:
            cur.execute(create_table_query)
            conn.commit()  # Commit the changes for table creation
        print("Table 'employees' checked/created successfully.")
    except psycopg2.Error as e:
        print(f"Error creating table: {e}")
        conn.rollback()  # Rollback in case of error
        sys.exit(1)


def insert_single_employee(conn, first_name, last_name, email):
    """Inserts a single employee record into the 'employees' table."""
    insert_query = """
    INSERT INTO employees (first_name, last_name, email)
    VALUES (%s, %s, %s) RETURNING id;
    """
    employee_id = None
    try:
        with conn.cursor() as cur:
            cur.execute(insert_query, (first_name, last_name, email))
            employee_id = cur.fetchone()[0]  # Get the ID of the newly inserted row
            conn.commit()  # Commit the transaction
            print(
                f"Employee '{first_name} {last_name}' inserted successfully with ID: {employee_id}."
            )
    except psycopg2.IntegrityError as e:  # Catch potential errors like duplicate email
        print(f"Integrity error inserting employee '{first_name} {last_name}': {e}")
        conn.rollback()  # Rollback the transaction
    except psycopg2.Error as e:
        print(f"Error inserting single employee '{first_name} {last_name}': {e}")
        conn.rollback()  # Rollback the transaction
    except Exception as e:
        print(f"An unexpected error occurred during single insert: {e}")
        conn.rollback()
    return employee_id


def insert_multiple_employees(conn, employee_data):
    """
    Inserts multiple employee records into the 'employees' table.
    employee_data should be a list of tuples, e.g.,
    [('Alice', 'Smith', 'alice.smith@example.com'), ('Bob', 'Johnson', 'bob.j@example.com')]
    """
    insert_query = """
    INSERT INTO employees (first_name, last_name, email)
    VALUES (%s, %s, %s);
    """
    try:
        with conn.cursor() as cur:
            cur.executemany(insert_query, employee_data)
            conn.commit()  # Commit the transaction
            print(f"Successfully inserted {cur.rowcount} employees.")
    except psycopg2.IntegrityError as e:
        print(f"Integrity error inserting multiple employees: {e}")
        conn.rollback()
    except psycopg2.Error as e:
        print(f"Error inserting multiple employees: {e}")
        conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred during multiple insert: {e}")
        conn.rollback()

In [5]:
if __name__ == "__main__":
    conn = None
    try:
        # Establish database connection
        conn = connect_to_db()

        # Ensure the table exists (optional, good for testing)
        create_table_if_not_exists(conn)

        # Example: Insert a single employee
        insert_single_employee(conn, "John", "Doe", "john.doe@example.com")
        insert_single_employee(
            conn, "Jane", "Doe", "jane.doe@example.com"
        )  # Another single insert

        # Example: Insert an employee that might cause an integrity error (if email is unique)
        insert_single_employee(conn, "John", "Dorian", "john.doe@example.com")

        # Example: Insert multiple employees
        employees_to_add = [
            ("Peter", "Pan", "peter.pan@example.com"),
            ("Wendy", "Darling", "wendy.darling@example.com"),
            ("Captain", "Hook", "hook@example.org"),
        ]
        insert_multiple_employees(conn, employees_to_add)

        # Example: Another batch, one of which might be a duplicate
        more_employees = [
            ("Tinker", "Bell", "tinker.bell@example.com"),
            ("Smee", "Pirate", "hook@example.org"),  # Potential duplicate email
        ]
        insert_multiple_employees(conn, more_employees)

    except Exception as e:
        print(f"A critical error occurred in the main execution: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

Table 'employees' checked/created successfully.
Employee 'John Doe' inserted successfully with ID: 1.
Employee 'Jane Doe' inserted successfully with ID: 2.
Integrity error inserting employee 'John Dorian': duplicate key value violates unique constraint "employees_email_key"
DETAIL:  Key (email)=(john.doe@example.com) already exists.

Successfully inserted 3 employees.
Integrity error inserting multiple employees: duplicate key value violates unique constraint "employees_email_key"
DETAIL:  Key (email)=(hook@example.org) already exists.

Database connection closed.


In [6]:
def read_all_employees(conn):
    """Reads all records from the 'employees' table and prints them."""
    select_query = (
        "SELECT id, first_name, last_name, email, hire_date FROM employees ORDER BY id;"
    )
    try:
        with conn.cursor() as cur:
            cur.execute(select_query)
            rows = cur.fetchall()  # Fetches all rows from the query result

            if not rows:
                print("\nNo employees found in the table.")
                return

            print("\n--- All Employees ---")
            for row in rows:
                # Each 'row' is a tuple representing a database record
                # (id, first_name, last_name, email, hire_date)
                print(
                    f"ID: {row[0]}, Name: {row[1]} {row[2]}, Email: {row[3]}, Hired: {row[4]}"
                )
            print("--------------------")
            return (
                rows  # Optionally return the rows if you need to process them further
            )
    except psycopg2.Error as e:
        print(f"Error reading employees: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during read: {e}")
    return None  # Return None in case of an error


# --- NEW FUNCTION TO READ DATA BY EMAIL (Example of conditional read) ---
def read_employee_by_email(conn, email_address):
    """Reads an employee record from the 'employees' table based on email."""
    select_query = "SELECT id, first_name, last_name, email, hire_date FROM employees WHERE email = %s;"
    try:
        with conn.cursor() as cur:
            cur.execute(
                select_query, (email_address,)
            )  # Pass email as a tuple for the placeholder
            row = cur.fetchone()  # Fetches the first matching row

            if row:
                print(f"\n--- Employee Found by Email ({email_address}) ---")
                print(
                    f"ID: {row[0]}, Name: {row[1]} {row[2]}, Email: {row[3]}, Hired: {row[4]}"
                )
                print("--------------------------------------")
                return row
            else:
                print(f"\nNo employee found with email: {email_address}")
                return None
    except psycopg2.Error as e:
        print(f"Error reading employee by email '{email_address}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred while reading by email: {e}")
    return None

In [8]:
if __name__ == "__main__":
    conn = None
    try:
        # Establish database connection
        conn = connect_to_db()

        # --- READ DATA ---
        # Read and print all employees
        all_employees_data = read_all_employees(conn)

        # Example: Read a specific employee
        read_employee_by_email(conn, "jane.doe@example.com")
        read_employee_by_email(conn, "nonexistent.email@example.com")

    except Exception as e:
        print(f"A critical error occurred in the main execution: {e}")
    finally:
        if conn:
            conn.close()
            print("\nDatabase connection closed.")


--- All Employees ---
ID: 1, Name: John Doe, Email: john.doe@example.com, Hired: 2025-05-13
ID: 2, Name: Jane Doe, Email: jane.doe@example.com, Hired: 2025-05-13
ID: 4, Name: Peter Pan, Email: peter.pan@example.com, Hired: 2025-05-13
ID: 5, Name: Wendy Darling, Email: wendy.darling@example.com, Hired: 2025-05-13
ID: 6, Name: Captain Hook, Email: hook@example.org, Hired: 2025-05-13
--------------------

--- Employee Found by Email (jane.doe@example.com) ---
ID: 2, Name: Jane Doe, Email: jane.doe@example.com, Hired: 2025-05-13
--------------------------------------

No employee found with email: nonexistent.email@example.com

Database connection closed.
